[Dataset](https://www.kaggle.com/datasets/zhangluyuan/ab-testing) was downloaded from Kaggle.

In [49]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math

df=pd.read_csv('ab_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [4]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


Need to check that the old page is shown to users from the control group, and the new page is shown to users from the treatment one. If there is mismatch between groups, then these observations must be excluded for correct analysis.

In [23]:
mask1=(df['group']=='control')&(df['landing_page']=='new_page')
index_to_drop1=df[mask1].index
df=df.drop(index_to_drop1)

In [24]:
mask2=(df['group']=='treatment')&(df['landing_page']=='old_page')
index_to_drop2=df[mask2].index
df=df.drop(index_to_drop2)

In [26]:
print(df.shape)
df['group'].value_counts()

(290585, 5)


treatment    145311
control      145274
Name: group, dtype: int64

In [27]:
#Check how many duplicated users exist
print(df['user_id'].count())
print(df['user_id'].nunique())

290585
290584


In [28]:
#Drop duplicated users
df.drop_duplicates(subset='user_id', keep='first', inplace=True)

In [33]:
#Show the % split between users who saw new vs old page
#Calculate pooled probability
mask3=df['group']=='control'
conversions_control=df['converted'][mask3].sum()
total_users_control=df['converted'][mask3].count()

mask4=df['group']=='treatment'
conversions_treatment=df['converted'][mask4].sum()
total_users_treatment=df['converted'][mask4].count()

In [34]:
print ('Split of control users who saw old page vs treatment users who saw new page: ',
      round(total_users_control/df['converted'].count()*100, 2), '%',
      round(total_users_treatment/df['converted'].count()*100, 2), '%',)

Split of control users who saw old page vs treatment users who saw new page:  49.99 % 50.01 %


In [35]:
#Count number of users who converted in each group
print('Number of control users who converted on old page: ', conversions_control)
print('Percentage of control users who converted: ', round((conversions_control/total_users_control)*100, 2),'%')

Number of control users who converted on old page:  17489
Percentage of control users who converted:  12.04 %


In [37]:
print('Number of treatment users who converted on new page: ', conversions_treatment)
print('Percentage of treatment users who converted: ', round((conversions_treatment/total_users_treatment)*100, 2),'%')

Number of treatment users who converted on new page:  17264
Percentage of treatment users who converted:  11.88 %


### Test

In [38]:
baseline_rate=conversions_control/total_users_control
baseline_rate

0.1203863045004612

In [39]:
practical_significance=0.01 #User defined

In [40]:
confidence_level=0.05 #User defined, for a 95% interval

In [41]:
sensitivity=0.8 #User defined

In [43]:
effect_size=sms.proportion_effectsize(baseline_rate, baseline_rate+practical_significance)
effect_size

-0.03020276534775579

In [45]:
sample_size=sms.NormalIndPower().solve_power(effect_size, power=sensitivity, alpha=confidence_level, ratio=1)
print('Required sample size: ', round(sample_size), 'per group')

Required sample size:  17209 per group


In [46]:
mask5=df['group']=='control'
conversions_control=df['converted'][mask5].sum()
total_users_control=df['converted'][mask5].count()

mask6=df['group']=='treatment'
conversions_treatment=df['converted'][mask6].sum()
total_users_treatment=df['converted'][mask6].count()

In [47]:
prob_pooled=(conversions_control+conversions_treatment)/(total_users_control+total_users_treatment)

In [50]:
se_pooled=math.sqrt(prob_pooled*(1-prob_pooled)*(1/total_users_control+1/total_users_treatment))

In [51]:
z_score=stats.norm.ppf(1-confidence_level/2)

In [52]:
margin_of_error=se_pooled*z_score

In [53]:
#Calculate dhat, the estimated difference between probability of conversions in the experiment and control groups
d_hat=(conversions_treatment/total_users_treatment)-(conversions_control/total_users_control)

In [54]:
#Test if we can reject the null hypothese
lower_bound=d_hat-margin_of_error
upper_bound=d_hat+margin_of_error

In [55]:
if practical_significance<lower_bound:
    print('Reject the null hypothese')
else:
    print('Do not reject the null hypothese')
print ('The lower bound of the confidence interval is: ', round (lower_bound*100,2), '%')
print ('The upper bound of the confidence interval is: ', round (upper_bound*100,2), '%')

Do not reject the null hypothese
The lower bound of the confidence interval is:  -0.39 %
The upper bound of the confidence interval is:  0.08 %


#### Conclusion
There is no statistically significant difference between the two samples. The change that was made to the page had no effect. So we can test the following hypothesis.